In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from collections import namedtuple

import matplotlib.pyplot as plt

from src.data.filesystem import fopen
from src.data.utils import load_train_test
from src.models.triplet_loss import get_near_negatives

In [ ]:
given_surname = "given"
size = "tiny"
Config = namedtuple("Config", "train_path near_negatives_path")
config = Config(
    train_path=f"s3://familysearch-names/processed/tree-hr-{given_surname}-similar-train-{size}.csv.gz",
    near_negatives_path=f"s3://nama-data/data/processed/tree-hr-{given_surname}-near-negatives-{size}.csv.gz",
)

In [ ]:
[train] = load_train_test([config.train_path])

input_names_train, weighted_actual_names_train, candidate_names_train = train

In [ ]:
threshold = 0.001
filtered_wans_train = []
for wan in weighted_actual_names_train:
    wans = [(name, weight, freq) for name, weight, freq in wan if weight > threshold]
    filtered_wans_train.append(wans)
print("filtered", sum(len(wan) for wan in filtered_wans_train))

In [ ]:
k = 50
lower_threshold = 0.6
upper_threshold = 0.74
near_negatives_train = get_near_negatives(
    input_names_train, weighted_actual_names_train, candidate_names_train, 
    k=k, lower_threshold=lower_threshold, upper_threshold=upper_threshold
)

In [ ]:
print("input_names_train", len(input_names_train))
print("filtered_wans_train", sum(len(wan) for wan in filtered_wans_train))
print("candidate_names_train", len(candidate_names_train))
print("near_negatives_train", sum(len(negs) for _, negs in near_negatives_train.items()))

In [ ]:
ix = 123
print(input_names_train[ix])
wans = sorted(filtered_wans_train[ix], key=lambda x: -x[2])
# print("positives", list(name for name, _, _ in weighted_actual_names_train[ix]))
print("positives", wans)
print("negatives", near_negatives_train[input_names_train[ix]])

In [ ]:
weights = []
for wan in weighted_actual_names_train:
    for _, weight, _ in wan:
        weights.append(weight)

In [ ]:
ax = plt.gca()
ax.set_ylim([0, 10000])
ax.set_xlim([0, 0.1])
plt.hist(weights, bins=100)